In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")

# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=None, end_row=None
# )
data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep="-", start_row=None, end_row=None
)
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" -> ", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: end-MY | Last line: start-NF

------

Convert to:

First line: ['end', 'MY']

Last line: ['start', 'NF']

------

In [4]:
connections = {}
small_cave_li = []
big_cave_li = []

for line in data:
    if "start" in line:
        if "start" not in connections:
            connections["start"] = [line[1]]
        else:
            connections["start"].append(line[1])
    elif "end" in line:
        if line[1] not in connections:
            connections[line[1]] = ["end"]
        else:
            connections[line[1]].append("end")
    else:
        if line[0] not in connections:
            connections[line[0]] = [line[1]]
        else:
            connections[line[0]].append(line[1])
        if line[1] not in connections:
            connections[line[1]] = [line[0]]
        else:
            connections[line[1]].append(line[0])

all_caves = list(connections.keys())
all_caves.remove("start")
for cave in all_caves:
    if cave.isupper():
        big_cave_li.append(cave)
    else:
        small_cave_li.append(cave)

In [13]:
%%time
def move_step_q1(path):
    current_cave = path[-1]
    if current_cave == "end":
        return []
    return [
        cave
        for cave in connections[current_cave]
        if (cave == "end")
        or (cave in big_cave_li)
        or ((cave in small_cave_li) and (cave not in path))
    ]


from collections import Counter


def move_step_q2(path):
    current_cave = path[-1]
    if current_cave == "end":
        return []
    return [
        cave
        for cave in connections[current_cave]
        if (cave == "end")
        or (cave in big_cave_li)
        or (
            (cave in small_cave_li)
            and (
                (cave not in path)
                or (
                    len(np.intersect1d(path, small_cave_li)) == 0
                    or (
                        Counter([i for i in path if i in small_cave_li]).most_common(1)[
                            0
                        ][1]
                        < 2
                    )
                )
            )
        )
    ]


def count_path(record_paths, move_fun):
    flag = True
    while flag:
        flag = False
        new_record_paths = []
        for path in record_paths:
            next_caves = move_fun(path)
            if len(next_caves) == 0:
                continue
            flag = True
            org_path = path[:]
            path.append(next_caves[0])
            if len(next_caves) > 1:
                for idx in range(len(next_caves) - 1):
                    new_path = org_path[:]
                    new_path.append(next_caves[idx + 1])
                    new_record_paths.append(new_path)

        record_paths.extend(new_record_paths)

    count = 0
    for path in record_paths:
        if path[-1] == "end":
            count += 1
    return count


record_paths = [["start"]]
print(f"Answer to Q1: {count_path(record_paths, move_step_q1)}")
record_paths = [["start"]]
print(f"Answer to Q2: {count_path(record_paths, move_step_q2)}")

Answer to Q1: 5076

Answer to Q2: 145643

Wall time: 1min 32s


In [14]:
%%time
def move_step_q1(path):
    current_cave = path[-1]
    if current_cave == "end":
        return []
    return [
        cave
        for cave in connections[current_cave]
        if (cave == "end")
        or (cave in big_cave_li)
        or ((cave in small_cave_li) and (cave not in path))
    ]


def move_step_q2(path):
    current_cave = path[-1]
    if current_cave == "end":
        return []
    return [
        cave
        for cave in connections[current_cave]
        if (cave == "end")
        or (cave in big_cave_li)
        or (
            (cave in small_cave_li)
            and (
                (cave not in path)
                or (
                    len(np.intersect1d(path, small_cave_li))==len([i for i in path if i in small_cave_li])
                )
            )
        )
    ]


def count_path(record_paths, move_fun):
    flag = True
    while flag:
        flag = False
        new_record_paths = []
        for path in record_paths:
            next_caves = move_fun(path)
            if len(next_caves) == 0:
                continue
            flag = True
            org_path = path[:]
            path.append(next_caves[0])
            if len(next_caves) > 1:
                for idx in range(len(next_caves) - 1):
                    new_path = org_path[:]
                    new_path.append(next_caves[idx + 1])
                    new_record_paths.append(new_path)

        record_paths.extend(new_record_paths)

    count = 0
    for path in record_paths:
        if path[-1] == "end":
            count += 1
    return count


record_paths = [["start"]]
print(f"Answer to Q1: {count_path(record_paths, move_step_q1)}")
record_paths = [["start"]]
print(f"Answer to Q2: {count_path(record_paths, move_step_q2)}")

Answer to Q1: 5076

Answer to Q2: 145643

Wall time: 1min 18s


In [12]:
%%time
def move_step_q1(path):
    current_cave = path[-1]
    if current_cave == "end":
        return []
    return [
        cave
        for cave in connections[current_cave]
        if (cave == "end")
        or (cave in big_cave_li)
        or ((cave in small_cave_li) and (cave not in path))
    ]


from collections import Counter


def move_step_q2(path):
    current_cave = path[-1]
    if current_cave == "end":
        return []
    return [
        cave
        for cave in connections[current_cave]
        if (cave == "end")
        or (cave in big_cave_li)
        or (
            (cave in small_cave_li)
            and (
                (cave not in path)
                or (
                    len(np.intersect1d(path, small_cave_li)) == 0
                    or (
                        Counter([i for i in path if i in small_cave_li]).most_common(1)[
                            0
                        ][1]
                        < 2
                    )
                )
            )
        )
    ]


def count_path(record_paths, move_fun):

    count = 0
    flag = True
    while flag:
        flag = False
        new_record_paths = []
        record_path_to_remove = []
        finished_path = []
        for path in record_paths:
            next_caves = move_fun(path)
            if len(next_caves) == 0:
                if path[-1] != "end":
                    record_path_to_remove.append(path)
                else:
                    finished_path.append(path)
                continue
            flag = True
            org_path = path[:]
            path.append(next_caves[0])
            if len(next_caves) > 1:
                for idx in range(len(next_caves) - 1):
                    new_path = org_path[:]
                    new_path.append(next_caves[idx + 1])
                    new_record_paths.append(new_path)

        record_paths.extend(new_record_paths)
        for path in [*record_path_to_remove, *finished_path]:
            record_paths.remove(path)
        count += len(finished_path)
    return count


record_paths = [["start"]]
print(f"Answer to Q1: {count_path(record_paths, move_step_q1)}")
record_paths = [["start"]]
print(f"Answer to Q2: {count_path(record_paths, move_step_q2)}")

Answer to Q1: 5076

Answer to Q2: 145643

Wall time: 3min 55s
